# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
# Load csv file ouput from WeatherPy
file = "../output_data/df_wx.csv"

#Create new dataframe from the above csv file
new_df_wx = pd.read_csv(file)
new_df_wx.head()

,Unnamed: 0,City,Lat,Lon,Country,Humidity,Max Temperature,Wind Speed,Cloudiness,Date
0,0,Shuyskoye,59.3736,41.0305,RU,95,33.49,8.88,96,1667836795
1,1,Hithadhoo,-0.6000,73.0833,MV,75,82.42,16.93,66,1667836797
2,2,Denpasar,-8.6500,115.2167,ID,88,80.26,4.61,20,1667836519
3,3,Kaitangata,-46.2817,169.8464,NZ,91,47.59,4.97,86,1667836799
4,4,Deputatsky,69.3000,139.9000,RU,88,-2.83,7.36,100,1667836536


In [25]:
new_df_wx.shape

(580, 10)

In [26]:
nullCount= new_df_wx.isna().sum()
print(nullCount)

Unnamed: 0         0
City               0
Lat                0
Lon                0
Country            6
Humidity           0
Max Temperature    0
Wind Speed         0
Cloudiness         0
Date               0
dtype: int64


In [27]:
nan_rows  = new_df_wx[new_df_wx.isna().any(axis=1)]
nan_rows

,Unnamed: 0,City,Lat,Lon,Country,Humidity,Max Temperature,Wind Speed,Cloudiness,Date
92,92,Lüderitz,-26.6481,15.1594,NaN,67,69.69,12.39,65,1667836778
275,275,Grootfontein,-19.5667,18.1167,NaN,20,83.61,11.52,54,1667837181
366,366,Keetmanshoop,-26.5833,18.1333,NaN,48,70.18,8.43,100,1667837306
393,393,Oranjemund,-28.5500,16.4333,NaN,71,67.14,11.63,34,1667837343
418,418,Walvis Bay,-22.9575,14.5053,NaN,72,66.09,11.95,38,1667837378
445,445,Rundu,-17.9333,19.7667,NaN,37,82.89,2.26,47,1667837415


In [28]:
clean_city_wx_df = new_df_wx.dropna()
clean_city_wx_df

,Unnamed: 0,City,Lat,Lon,Country,Humidity,Max Temperature,Wind Speed,Cloudiness,Date
0,0,Shuyskoye,59.3736,41.0305,RU,95,33.49,8.88,96,1667836795
1,1,Hithadhoo,-0.6000,73.0833,MV,75,82.42,16.93,66,1667836797
2,2,Denpasar,-8.6500,115.2167,ID,88,80.26,4.61,20,1667836519
3,3,Kaitangata,-46.2817,169.8464,NZ,91,47.59,4.97,86,1667836799
4,4,Deputatsky,69.3000,139.9000,RU,88,-2.83,7.36,100,1667836536
...,...,...,...,...,...,...,...,...,...,...
575,575,Aquin,18.2833,-73.4000,HT,64,82.33,3.49,2,1667837596
576,576,Nouadhibou,20.9310,-17.0347,MR,55,74.37,11.18,0,1667837597
577,577,Okhotsk,59.3833,143.3000,RU,89,8.19,11.30,65,1667837598
578,578,Acuexcomac,19.0333,-98.3833,MX,26,67.21,10.36,0,1667837600


In [29]:
nan_rows2  = new_df_wx[new_df_wx.isna().any(axis=1)]
nan_rows

,Unnamed: 0,City,Lat,Lon,Country,Humidity,Max Temperature,Wind Speed,Cloudiness,Date
92,92,Lüderitz,-26.6481,15.1594,NaN,67,69.69,12.39,65,1667836778
275,275,Grootfontein,-19.5667,18.1167,NaN,20,83.61,11.52,54,1667837181
366,366,Keetmanshoop,-26.5833,18.1333,NaN,48,70.18,8.43,100,1667837306
393,393,Oranjemund,-28.5500,16.4333,NaN,71,67.14,11.63,34,1667837343
418,418,Walvis Bay,-22.9575,14.5053,NaN,72,66.09,11.95,38,1667837378
445,445,Rundu,-17.9333,19.7667,NaN,37,82.89,2.26,47,1667837415


In [30]:
nullCount2= clean_city_wx_df.isna().sum()
print(nullCount2)

Unnamed: 0         0
City               0
Lat                0
Lon                0
Country            0
Humidity           0
Max Temperature    0
Wind Speed         0
Cloudiness         0
Date               0
dtype: int64


In [31]:
"""
Null drop of 6 countries verified
"""

'\nNull drop of 6 countries verified\n'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [33]:
# Using Latitude and Longitude as locations and Humidity as the weight
locations = clean_city_wx_df[["Lat", "Lon"]]

# Using humidity as weight and converting the column to a float data type
humidity = clean_city_wx_df["Humidity"].astype(float)

In [38]:
# Configure map
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=5)

# Add layer
fig.add_layer(heatmap_layer)

# To display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# New DataFrame to match Ideal Wx conditions: 

# A max temperature lower than 80 degrees but higher than 70
temp_max_df = clean_city_wx_df[(clean_city_wx_df["Max Temperature"] > 70) & (clean_city_wx_df["Max Temperature"] < 80)]

# Wind speed less than 10 mph
wind_velocity_df = temp_max_df[temp_max_df["Wind Speed"] < 10]

# Zero cloudiness
z_cloud_df = wind_velocity_df[wind_velocity_df["Cloudiness"] == 0]
z_cloud_df = z_cloud_df.dropna()
z_cloud_df

,Unnamed: 0,City,Lat,Lon,Country,Humidity,Max Temperature,Wind Speed,Cloudiness,Date
100,100,Mailsi,29.8003,72.1758,PK,47,71.69,7.47,0,1667836942
183,183,Avera,33.1940,-82.5271,US,62,77.56,5.75,0,1667837057
227,227,Brookhaven,31.5791,-90.4407,US,75,78.46,4.45,0,1667837116
283,283,San Vicente,-26.6167,-54.1333,AR,29,76.86,4.90,0,1667837192
297,297,Aswān,24.0934,32.9070,EG,18,76.33,9.55,0,1667837211
340,340,Bandarbeyla,9.4942,50.8122,SO,69,79.56,9.51,0,1667837270
441,441,Najrān,17.4924,44.1277,SA,20,73.20,8.14,0,1667837410
529,529,Colorado,-22.8375,-51.9731,BR,34,78.57,6.69,0,1667837532


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
# Store into variable named hotel_df
hotel_df = z_cloud_df

In [16]:
# Add a "Hotel Name" column to the DataFrame
hotel_df.insert(1,"Hotel Name", "")
hotel_df

,Unnamed: 0,Hotel Name,City,Lat,Lon,Country,Humidity,Max Temperature,Wind Speed,Cloudiness,Date
100,100,,Mailsi,29.8003,72.1758,PK,47,71.69,7.47,0,1667836942
183,183,,Avera,33.1940,-82.5271,US,62,77.56,5.75,0,1667837057
227,227,,Brookhaven,31.5791,-90.4407,US,75,78.46,4.45,0,1667837116
283,283,,San Vicente,-26.6167,-54.1333,AR,29,76.86,4.90,0,1667837192
297,297,,Aswān,24.0934,32.9070,EG,18,76.33,9.55,0,1667837211
340,340,,Bandarbeyla,9.4942,50.8122,SO,69,79.56,9.51,0,1667837270
441,441,,Najrān,17.4924,44.1277,SA,20,73.20,8.14,0,1667837410
529,529,,Colorado,-22.8375,-51.9731,BR,34,78.57,6.69,0,1667837532


In [18]:
hotels = []

# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the rows
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lon = row["Lon"]
    
    params["location"] = f"{lat},{lon}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and get the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Hotel not found... skipping.")
    

    
hotel_df


Hotel not found... skipping.


,Unnamed: 0,Hotel Name,City,Lat,Lon,Country,Humidity,Max Temperature,Wind Speed,Cloudiness,Date
100,100,Mian Lodge,Mailsi,29.8003,72.1758,PK,47,71.69,7.47,0,1667836942
183,183,,Avera,33.1940,-82.5271,US,62,77.56,5.75,0,1667837057
227,227,Americas Best Value Inn Brookhaven,Brookhaven,31.5791,-90.4407,US,75,78.46,4.45,0,1667837116
283,283,Complejo Turístico Farelluk,San Vicente,-26.6167,-54.1333,AR,29,76.86,4.90,0,1667837192
297,297,Nuba Nile,Aswān,24.0934,32.9070,EG,18,76.33,9.55,0,1667837211
340,340,JABIR HOTEL,Bandarbeyla,9.4942,50.8122,SO,69,79.56,9.51,0,1667837270
441,441,محمد هادي ال فطيح,Najrān,17.4924,44.1277,SA,20,73.20,8.14,0,1667837410
529,529,SÍTIO BOM JESUS,Colorado,-22.8375,-51.9731,BR,34,78.57,6.69,0,1667837532


In [19]:
hotel_df.head()

,Unnamed: 0,Hotel Name,City,Lat,Lon,Country,Humidity,Max Temperature,Wind Speed,Cloudiness,Date
100,100,Mian Lodge,Mailsi,29.8003,72.1758,PK,47,71.69,7.47,0,1667836942
183,183,,Avera,33.1940,-82.5271,US,62,77.56,5.75,0,1667837057
227,227,Americas Best Value Inn Brookhaven,Brookhaven,31.5791,-90.4407,US,75,78.46,4.45,0,1667837116
283,283,Complejo Turístico Farelluk,San Vicente,-26.6167,-54.1333,AR,29,76.86,4.90,0,1667837192
297,297,Nuba Nile,Aswān,24.0934,32.9070,EG,18,76.33,9.55,0,1667837211


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [22]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))